# NetCDF Explore

In [1]:
import glob
import re
from netCDF4 import Dataset

import pandas as pd

from os.path import join

## Get Files

In [2]:
file_dir = '/home/squirt/Documents/data/rp_weather_data/scratch/frme/caramel_for_zenodo/20200102'
to_get_path = join(file_dir, "**/*.nc")
nc_files = glob.glob(to_get_path, recursive=True)

In [3]:
len(nc_files)

15360

In [4]:
df = pd.DataFrame(nc_files, columns=['file_path'])

# ChatGPT (so ???)
def extract_parts(filename):
    filename = re.sub(r'\.nc$', '', filename)
    # Split the string on _
    filename_parts = re.split('_', filename)
    filename_parts.remove('hence')
    return filename_parts

# Assuming df is your DataFrame and 'file_path' is the column with the filenames
df[['date_time', 'region', 'grid_length', 'config', 'region_size', 'sampling', 'time', 'processing', 'code']] = df['file_path'].apply(lambda x: pd.Series(extract_parts(x.split('/')[-1])))


In [5]:
extract_parts(nc_files[0])

['/home/squirt/Documents/data/rp',
 'weather',
 'data/scratch/frme/caramel',
 'for',
 'zenodo/20200102/20200102T0000Z',
 'r13',
 'km1p5',
 'RA2T',
 '224x224sampling',
 '2x2',
 'time018',
 'AVG',
 '16004']

In [6]:
df.head()

,file_path,date_time,region,grid_length,config,region_size,sampling,time,processing,code
0,/home/squirt/Documents/data/rp_weather_data/sc...,20200102T0000Z,r13,km1p5,RA2T,224x224sampling,2x2,time018,AVG,16004
1,/home/squirt/Documents/data/rp_weather_data/sc...,20200102T0000Z,r34,km1p5,RA2T,224x224sampling,2x2,time005,STD,16004
2,/home/squirt/Documents/data/rp_weather_data/sc...,20200102T0000Z,r32,km1p5,RA2T,224x224sampling,2x2,time023,AVG,00408
3,/home/squirt/Documents/data/rp_weather_data/sc...,20200102T0000Z,r75,km1p5,RA2T,224x224sampling,2x2,time001,STD,00033
4,/home/squirt/Documents/data/rp_weather_data/sc...,20200102T0000Z,r08,km1p5,RA2T,224x224sampling,2x2,time011,AVG,00408


In [7]:
simple_df = df[['region', 'time', 'code', 'file_path', 'processing']]
df_big_group = simple_df.groupby(['region', 'time', 'code'])

In [11]:
result = []
for name, group in df_big_group:
    region, time, code = name
    file_dict = {}
    for index, row in group.iterrows():
        file_dict[row['processing']] = row['file_path']

    result.append({
        'region': region,
        'time': time,
        'code': code,
        'files': file_dict
    })

In [12]:
result[0]

{'region': 'r01',
 'time': 'time000',
 'code': '00010',
 'files': {'STD': '/home/squirt/Documents/data/rp_weather_data/scratch/frme/caramel_for_zenodo/20200102/20200102T0000Z_r01_km1p5_RA2T_224x224sampling_hence_2x2_time000_STD_00010.nc',
  'AVG': '/home/squirt/Documents/data/rp_weather_data/scratch/frme/caramel_for_zenodo/20200102/20200102T0000Z_r01_km1p5_RA2T_224x224sampling_hence_2x2_time000_AVG_00010.nc'}}